In [1]:
import pandas as pd
import ast
import numpy as np

In [64]:
def state_merge(path_to_data,path_to_new):
    df = pd.read_csv(path_to_data,index_col='Unnamed: 0')
    df['census'] = df['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
    df['county'] = df['census'].apply(lambda x: x[:5])
    df['state'] = df['county'].apply(lambda x: x[:2])
    state_df = df.groupby('state').mean()
    state_df=state_df[['comp','LILATracts_1And10']]
    state_df = state_df.join(state_data)
    #state_df.to_csv(path_to_new)
    return state_df

In [71]:
def county_merge(path_to_data,path_to_new):
    df = pd.read_csv(path_to_data,index_col='Unnamed: 0')
    df['census'] = df['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
    df['county'] = df['census'].apply(lambda x: x[:5])
    df['state'] = df['county'].apply(lambda x: x[:2])
    county_df = df.groupby('county').mean()
    county_df=county_df[['comp','LILATracts_1And10']]
    county_df = county_df.join(state_data)
    county_df.to_csv(path_to_new)

In [4]:
#read in target data
target = pd.read_csv('../data/target.csv', index_col=0)
target['CensusTract'] = target['CensusTract'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
target.set_index("CensusTract", inplace =True)

In [5]:
target.head()

,LILATracts_1And10
CensusTract,
01001020100,0
01001020200,0
01001020300,0
01001020400,0
01001020500,0


### HEALTHY FOODS

In [24]:
healthy = pd.read_csv('../data/twitter_mongo/healthy_final.csv',index_col='Unnamed: 0')
healthy['census'] = healthy['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
healthy['county'] = healthy['census'].apply(lambda x: x[:5])
healthy['state'] = healthy['county'].apply(lambda x: x[:2])
#healthy.groupby('LILATracts_1And10').mean()['comp']

In [66]:
healthy[np.array([len(str(i))!=5 for i in healthy.county])]

,keyword,text,lat,long,census,comp,category,LILATracts_1And10,county,state


In [72]:
healthy.join()

,keyword,text,lat,long,census,comp,category,LILATracts_1And10,county,state
0,almond,"Almond cake , mango and raspberry compote , le...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12
1,avocado,Breakfast all day : On the Mix Slam with avoca...,25.752978,-80.261223,12086006206,-0.4389,healthy,0,12086,12
2,mango,"Almond cake , mango and raspberry compote , le...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12
3,raspberry,"Almond cake , mango and raspberry compote , le...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12
4,raspberry,"Dulcey , raspberry and coconut entremet at @ba...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12


In [68]:
county_sent = healthy.groupby('county').mean()

In [69]:
county_sent=county_sent[['comp','LILATracts_1And10']]

In [70]:
county_sent.head()

,comp,LILATracts_1And10
county,,
01003,0.265528,0.034483
01031,0.000000,0.000000
01065,0.000000,0.000000
01069,-0.127267,0.666667
01073,0.217862,0.500000


In [32]:
state_data = pd.read_csv('../data/state_data.csv',index_col=0)

In [47]:
state_data['FIPS'] = state_data['FIPS'].apply(lambda x: "0"+str(x) if len(str(x))==1 else str(x)) 
state_data.set_index('FIPS',inplace=True)


In [49]:
merged_state = state_sent.join(state_data)

In [51]:
merged_state.to_csv('../data/state_sent.csv')

In [74]:
merged_state['healthy_sent']=merged_state['comp']

In [111]:
unhealthy = pd.read_csv('../data/twitter_mongo/ff_stores_final.csv',index_col='Unnamed: 0')
unhealthy['census'] = unhealthy['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
unhealthy['county'] = unhealthy['census'].apply(lambda x: x[:5])
unhealthy['state'] = unhealthy['county'].apply(lambda x: x[:2])
state_unhealthy = unhealthy.groupby('state').mean()
state_unhealthy=state_unhealthy['comp']
#state_unhealthy = state_unhealthy.join(state_data)
#state_unhealthy.to_csv('../data/state_ff.csv')

In [112]:
merged_state3 = merged_state2.join(state_unhealthy)

In [113]:
merged_state3.fillna(merged_state3['comp'].mean(),inplace=True)

In [122]:
merged_state3.head()

,LILATracts_1And10,abbreviation,name,healthy_sent,grocery_sent,unhealthy_sent,ff_sent
state,,,,,,,
01,0.217742,AL,ALABAMA,0.200741,0.159634,0.287100,0.327672
02,0.166667,AK,ALASKA,0.368133,0.159634,0.056150,0.336106
04,0.186747,AZ,ARIZONA,0.328105,0.299548,0.243346,0.288653
05,0.142857,AR,ARKANSAS,0.299902,0.159634,0.410703,0.327659
06,0.028638,CA,CALIFORNIA,0.262870,0.155814,0.287623,0.288852


In [121]:
merged_state3.rename(columns={'comp':'ff_sent'}, inplace=True)

In [145]:
arr=np.array(['unhealthy','healthy'])
final = arr[(np.array(merged_state3['grocery_sent']+merged_state3['healthy_sent'])>(merged_state3['ff_sent']+merged_state3['unhealthy_sent']).values)*1]

In [123]:
max_val = merged_state3[['healthy_sent','grocery_sent','ff_sent','unhealthy_sent']].idxmax(axis=1)

In [125]:
merged_state3['max_sent']=max_val

In [146]:
merged_state3['final_sent']=final

In [147]:
merged_state3.to_csv('../data/grouped_state_sent.csv')

### UNHEALTHY FOODS

In [9]:
unhealthy = pd.read_csv('../data/twitter_mongo/unhealthy_final.csv',index_col='Unnamed: 0')

unhealthy.groupby('LILATracts_1And10').mean()['comp']

LILATracts_1And10
0    0.298508
1    0.310094
Name: comp, dtype: float64

### GROCERY STORES

In [10]:
grocery = pd.read_csv('../data/twitter_mongo/grocery_stores_final.csv',index_col='Unnamed: 0')

grocery.groupby('LILATracts_1And10').mean()['comp']

LILATracts_1And10
0    0.219386
1    0.432712
Name: comp, dtype: float64

In [52]:
unhealthy = pd.read_csv('../data/twitter_mongo/unhealthy_final.csv',index_col='Unnamed: 0')

In [53]:
unhealthy.county

0       39041
1       38035
2       38035
3        6059
4        6059
5        6037
6        6037
7        6037
8        6037
9       48029
10      48029
11      20137
12      20137
13      20137
14      20137
15      20137
16      20137
17      20137
18      20137
19      20137
20      20137
21      20137
22      20137
23      20137
24      20137
25      20137
26      20137
27      20137
28      20137
29      20137
        ...  
9586    37135
9587     6075
9588    17031
9589     6037
9590    45051
9591    21235
9592    47165
9593    13121
9594    12097
9595    36047
9596     5119
9597    53053
9598     6037
9599    24510
9600    27053
9601    19169
9602    13089
9603    22071
9604    34005
9605    26125
9606    27137
9607    21107
9608    26081
9609    48201
9610    42101
9611    42101
9612    24003
9613    26163
9614    17031
9615    17031
Name: county, Length: 9616, dtype: int64

In [54]:
unhealthy['census'] = unhealthy['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
unhealthy['county'] = unhealthy['census'].apply(lambda x: x[:5])
unhealthy['state'] = unhealthy['county'].apply(lambda x: x[:2])


### FAST FOODS

In [11]:
fastfood = pd.read_csv('../data/twitter_mongo/ff_stores_final.csv',index_col='Unnamed: 0')

fastfood.groupby('LILATracts_1And10').mean()['comp']

LILATracts_1And10
0    0.300494
1    0.344389
Name: comp, dtype: float64